In [7]:
import pandas as pd
from pathlib import Path
import hvplot.pandas
import numpy as np
# Imports for Alapaca SDK
import os
import requests
from dotenv import load_dotenv
import alpaca_trade_api as tradeapi
from alpaca_trade_api.rest import REST, TimeFrame

# Alpaca SDK Section

In [8]:
load_dotenv()

True

In [9]:
# define Alpaca Keys
alpaca_api_key = os.getenv('APCA_API_KEY_ID')
alpaca_secret_key = os.getenv('APCA_API_SECRET_KEY')
display(type(alpaca_api_key))
display(type(alpaca_secret_key))

str

str

In [10]:
# Create the Alpaca API object
alpaca = tradeapi.REST(
    alpaca_api_key,
    alpaca_secret_key,
    'https://data.alpaca.markets',
    api_version="v2")

In [11]:
# Get request for AMD closing data
amd_df = alpaca.get_bars("AMD", TimeFrame.Day, "2016-01-01", "2022-04-30", adjustment='raw').df


In [12]:
# Preview data frame
amd_df.head()

,open,high,low,close,volume,trade_count,vwap
timestamp,,,,,,,
2016-01-04 05:00:00+00:00,2.77,2.820,2.63,2.770,32516771,43861,2.714418
2016-01-05 05:00:00+00:00,2.77,2.800,2.64,2.750,12972260,27269,2.717732
2016-01-06 05:00:00+00:00,2.66,2.710,2.47,2.505,23759374,32861,2.577993
2016-01-07 05:00:00+00:00,2.41,2.480,2.26,2.275,22203514,37014,2.356621
2016-01-08 05:00:00+00:00,2.37,2.425,2.10,2.140,31822424,37159,2.194051


# Prep Dataframe

In [13]:
amd_df.index = pd.to_datetime(amd_df.index, format = '%Y/%m/%d').strftime('%Y-%m-%d')
amd_df = amd_df.drop(columns = ['open','high','low','volume','trade_count','vwap'])
amd_df.head()

,close
timestamp,
2016-01-04,2.770
2016-01-05,2.750
2016-01-06,2.505
2016-01-07,2.275
2016-01-08,2.140


In [16]:
amd_df = amd_df.rename(columns={"close": "Close"})
amd_df.describe()

,Close
count,1593.000000
mean,42.507021
std,39.168224
min,1.800000
25%,11.910000
50%,27.210000
75%,78.340000
max,161.910000


In [22]:
# Plot price action
amd_df.hvplot(y="Close")

:Curve   [timestamp]   (Close)

In [23]:
# Add the trade_type column to track buys and sells
amd_df['Trade_Type'] = np.nan
amd_df.head()

,Close,Trade_Type
timestamp,,
2016-01-04,2.770,NaN
2016-01-05,2.750,NaN
2016-01-06,2.505,NaN
2016-01-07,2.275,NaN
2016-01-08,2.140,NaN


# Global Variables

In [35]:
# Initialize initial capital, share size and accumulated shares
INITIAL_CAPITAL = 10000
SHARE_SIZE = 10
ACCUMULATED_SHARES = 0

# Variable to track the previous purchase price
PREV_BUY_PRICE = 0
PREV_SELL_PRICE = 0

# Moving Average Section

In [24]:
# Moving averages
short_window = 50
long_window = 100
ex_long_window = 200

amd_df['SMA50'] = amd_df['Close'].rolling(window=short_window).mean()
amd_df['SMA100'] = amd_df['Close'].rolling(window=long_window).mean()
amd_df['SMA200'] = amd_df['Close'].rolling(window=ex_long_window).mean()

# Create a column to hold the trading signal
amd_df["Signal"] = 0.0

In [26]:
# Generate the trading signal 0 or 1,
# where 1 is the short-window (SMA100) greater than the long-window (SMA200)
# and 0 is when the condition is not met
amd_df["Signal"][long_window:] = np.where(
    amd_df["SMA100"][long_window:] > amd_df["SMA200"][long_window:], 1.0, 0.0)

# Review the DataFrame
#amd_df.loc["2019-05-01":"2019-06-17"]

In [28]:
# Calculate the points in time when the Signal value changes
# Identify trade entry (1) and exit (-1) points
amd_df["Entry/Exit"] = amd_df["Signal"].diff()

# Review the DataFrame
#amd_df.loc["2019-11-01":"2019-12-17"]

In [29]:
# Plot the moving averages
amd_df.hvplot(x="timestamp",y=['SMA50','SMA100','SMA200'])

:NdOverlay   [Variable]
   :Curve   [timestamp]   (value)

In [30]:
# Visualize exit position relative to close price
exit = amd_df[amd_df['Entry/Exit'] == -1.0]['Close'].hvplot.scatter(
    color='yellow',
	marker="v",
    legend=False,
    ylabel='Price in $',
    width=1000,
    height=400)

# Show the plot
exit

# Visualize entry position relative to close price
entry = amd_df[amd_df['Entry/Exit'] == 1.0]['Close'].hvplot.scatter(
    color='purple',
	marker = "^",
    legend=False,
    ylabel='Price in $',
    width=1000,
    height=400)

# Show the plot
entry

# Visualize close price for the investment
security_close = amd_df[['Close']].hvplot(
    line_color='lightgray',
    ylabel='Price in $',
    width=1000,
    height=400)

# Show the plot
security_close

# Visualize moving averages
moving_avgs = amd_df[['SMA100', 'SMA200']].hvplot(
    ylabel='Price in $',
    width=1000,
    height=400)

# Show the plot
moving_avgs

:NdOverlay   [Variable]
   :Curve   [timestamp]   (value)

In [33]:
# Create the overlay plot
entry_exit_plot = security_close * moving_avgs * entry * exit

# Show the plot
entry_exit_plot.opts(
    title="AMD - SMA100, SMA200, Entry and Exit Points"
)

:Overlay
   .Curve.I          :Curve   [timestamp]   (Close)
   .NdOverlay.I      :NdOverlay   [Variable]
      :Curve   [timestamp]   (value)
   .Scatter.Close.I  :Scatter   [timestamp]   (Close)
   .Scatter.Close.II :Scatter   [timestamp]   (Close)

In [34]:
amd_df.loc['2018-06-10':'2018-06-20']

,Close,Trade_Type,SMA50,SMA100,SMA200,Signal,Entry/Exit
timestamp,,,,,,,
2018-06-11,15.73,NaN,11.8238,11.8365,11.90660,0.0,0.0
2018-06-12,15.85,NaN,11.9502,11.8703,11.92335,0.0,0.0
2018-06-13,16.32,NaN,12.0856,11.9076,11.94280,0.0,0.0
2018-06-14,16.25,NaN,12.2152,11.9436,11.96290,0.0,0.0
2018-06-15,16.34,NaN,12.3416,11.9776,11.98385,0.0,0.0
2018-06-18,17.11,NaN,12.4916,12.0216,12.00605,1.0,1.0
2018-06-19,16.69,NaN,12.6348,12.0644,12.02450,1.0,0.0
2018-06-20,16.52,NaN,12.7656,12.1001,12.04115,1.0,0.0


# Trading Logic

In [38]:
for index, row in amd_df.iterrows():
    if PREV_BUY_PRICE == 0:
        amd_df.loc[index, 'Trade_Type'] = 'Buy'
        amd_df.loc[index, 'Cost/Proceeds'] = -(row['Close'] * SHARE_SIZE)
        ACCUMULATED_SHARES += SHARE_SIZE
        PREV_BUY_PRICE = row['Close']
    else:
        amd_df.loc[index, 'Trade_Type'] = 'Hold'
    

In [42]:
amd_df
PREV_BUY_PRICE

2.77

In [41]:
# Loop through the Pandas DataFrame and initiate a trade each iteration
for index, row in btc_df.iterrows():
    if previous_purchase_price == 0: #& btc_df['Entry/Exit'] == float(1):
        btc_df.loc[index, 'Trade_Type'] = 'Buy'
        # Calculate cost of buy
        btc_df.loc[index, 'Cost/Proceeds'] = -(row['Close'] * share_size)
        # Add shares purchased to the number of accumulated shares
        accumulated_shares += share_size
    elif row['Close'] < previous_price:
        btc_df.loc[index, 'Trade_Type'] = 'Buy'
        # Calculate cost of buy
        btc_df.loc[index, 'Cost/Proceeds'] = -(row['Close'] * share_size)
        # Add shares purchased to the number of accumulated shares
        accumulated_shares += share_size
    elif row['Close'] > previous_price:
        btc_df.loc[index, 'Trade_Type'] = 'Sell'
        # Calculate proceeds of sell
        btc_df.loc[index, 'Cost/Proceeds'] = (row['Close'] * share_size)
        # Add shares purchased to the number of accumulated shares
        accumulated_shares -= share_size
    else:
        btc_df.loc[index, 'Trade_Type'] = 'Hold'
    
    # Update the previous_price to the current row's price
    previous_price = row["Close"]
    previous_purchase_price = row["Close"]
    
    # if the index is the last index of the DataFrame, sell
    # I THINK DELETE THIS IF PUT INTO PRACTICE
    if index == btc_df.index[-1]:
        btc_df.loc[index, "Trade_Type"] = "Sell"
        # Calculate proceeds of sell
        btc_df.loc[index, 'Cost/Proceeds'] = (row['Close'] * accumulated_shares)

In [43]:
display(accumulated_shares)
display(previous_purchase_price)
btc_df.head(10)

147600

39799.68

,Close,Trade_Type,SMA50,SMA100,SMA200,Signal,Entry/Exit,Cost/Proceeds
Date,,,,,,,,
2018-01-28,11786.30,Buy,NaN,NaN,NaN,0.0,NaN,-1178630.0
2018-01-29,11296.40,Buy,NaN,NaN,NaN,0.0,0.0,-1129640.0
2018-01-30,10106.30,Buy,NaN,NaN,NaN,0.0,0.0,-1010630.0
2018-01-31,10221.10,Sell,NaN,NaN,NaN,0.0,0.0,1022110.0
2018-02-01,9170.54,Buy,NaN,NaN,NaN,0.0,0.0,-917054.0
2018-02-02,8830.75,Buy,NaN,NaN,NaN,0.0,0.0,-883075.0
2018-02-03,9174.91,Sell,NaN,NaN,NaN,0.0,0.0,917491.0
2018-02-04,8277.01,Buy,NaN,NaN,NaN,0.0,0.0,-827701.0
2018-02-05,6955.27,Buy,NaN,NaN,NaN,0.0,0.0,-695527.0


# Profit/Loss Metrics

### Profit/Loss

In [16]:
# Calculate the total profit/loss for 100 share size orders
total_profit_loss = round(btc_df["Cost/Proceeds"].sum(), 2)
print(f"The total profit(-loss) is {total_profit_loss}")

The total profit(-loss) is 143494550.0


### Return on Investment

In [19]:
# Initialize the variable to hold the value of the invested capital
invested_capital = 0

# Calculate the invested capital by adding the cost of all buy trades
for index, row in btc_df.iterrows():
    if row["Trade_Type"] == "Buy":
        invested_capital = invested_capital + row["Cost/Proceeds"]


# Calculate the return on investment (ROI)
roi = round((total_profit_loss / -(invested_capital)) * 100, 2)

# Print the ROI
print(f"The trading algorithm resulted in a return on investment of {roi}%")

The trading algorithm resulted in a return on investment of 9.59%
